In [4]:
import ee
import geemap
import geopandas as gpd


In [2]:
ee.Authenticate()
ee.Initialize()

In [7]:
my_gpkg = gpd.read_file('/mnt/c/tmp/output/detail_1.gpkg').to_crs(epsg=4326)
bbox = my_gpkg.total_bounds

In [29]:
my_gpkg = gpd.read_file('/mnt/c/tmp/output/detail_1.gpkg').to_crs(epsg=4326)
bbox = my_gpkg.total_bounds

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2015-01-01', '2024-04-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)
ndvi_dataset = dataset.map(add_ndvi)
ndvi_clipped = ndvi_dataset.select('NDVI').median().clip(ee.Geometry.Rectangle([bbox[0], bbox[1], bbox[2], bbox[3]]))

# Convert floating point NDVI to integral by scaling up by 10000
ndvi_scaled = ndvi_clipped.multiply(10000).int()

# Reduce to vectors
pixel_scale = 10
vector_pixels = ndvi_scaled.reduceToVectors(
    geometry=ee.Geometry.Rectangle([bbox[0], bbox[1], bbox[2], bbox[3]]),
    crs=ndvi_scaled.projection(),
    scale=pixel_scale,
    geometryType='centroid',
)

pixel_centers = ee.FeatureCollection(vector_pixels)

# Display results
ndvi_params = {'min': 0.2 * 10000, 'max': 1 * 10000, 'palette': ['brown', 'yellow', 'green']}

latitude = (bbox[1] + bbox[3]) / 2
longitude = (bbox[0] + bbox[2]) / 2


visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map(center=[latitude, longitude], zoom=zoom_level) 
m.add_layer(dataset.mean(), visualization, 'RGB')
m.addLayer(ndvi_scaled, ndvi_params, 'NDVI')
m.addLayer(pixel_centers, point_params, 'Pixel Centers')
m


In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.first(),
                                  scale=500,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):

    def reduce_region_function(img):

        stat = img.reduceRegion(
            reducer=reducer,
            geometry=geometry,
            scale=scale,
            crs=crs,
            bestEffort=bestEffort,
            maxPixels=maxPixels,
            tileScale=tileScale)

        return ee.Feature(geometry, stat).set({'millis': img.date().millis()})

    return reduce_region_function

study_boundary_gdf = gpd.read_file("data/study_boundaries/extent.gpkg")

study_boundary_gdf = study_boundary_gdf.to_crs(epsg=4326)

# Define the time range
start_date = '2001-01-01'
end_date = '2023-04-30'

bounds = study_boundary_gdf.total_bounds
minx, miny, maxx, maxy = bounds

bbox = ee.Geometry.Rectangle([minx, miny, maxx, maxy])


# Map this function over the NDVI collection
collection_with_fire = collection_modis.map(process_fire_for_ndvi)


pois = []
with fiona.open('data/modis_centers_clipped2000.gpkg') as layer:
    for feature in layer:
        pnt = feature['geometry']['coordinates']
        pois.append(ee.Geometry.Point(*pnt))

def process_poi(poi):
    reduce_to_point = create_reduce_region_function(geometry=poi, scale=500)
    
    # Use the collection_with_fire here since it has the FirePresence band
    stat_fc = ee.FeatureCollection(collection_with_fire.map(reduce_to_point))
    features = stat_fc.getInfo()['features']
    lon, lat = poi.coordinates().getInfo()
    ndvi_data = []
    fire_data = []

    for feature in features:
        properties = feature['properties']
        millis = properties.get('millis')
        date = datetime.datetime.fromtimestamp(millis/1000).strftime('%Y-%m-%d')
        
        ndvi_value = properties.get('EVI')
        if ndvi_value is not None:
            ndvi = ndvi_value * 0.0001
        else:
            ndvi = None  
        
        fire_presence = properties.get('FirePresence', 0)

        ndvi_data.append((date, ndvi))
        fire_data.append((date, fire_presence))

    ndvi_df = pd.DataFrame(ndvi_data, columns=['Date', 'NDVI'])
    ndvi_df['Date'] = pd.to_datetime(ndvi_df['Date'])
    ndvi_df = ndvi_df.sort_values('Date')
    ndvi_max = ndvi_df.groupby(ndvi_df['Date'].dt.strftime('%Y-%m'))['NDVI'].max().to_dict()
    
    fire_df = pd.DataFrame(fire_data, columns=['Date', 'FirePresence'])
    fire_df['Date'] = pd.to_datetime(fire_df['Date'])
    fire_df = fire_df.sort_values('Date')
    fire_max = fire_df.groupby(fire_df['Date'].dt.strftime('%Y-%m'))['FirePresence'].max().to_dict()  # assuming you want the max for FirePresence
    
    return ({"EVI": ndvi_max, "Fire": fire_max, 'lon': lon, 'lat': lat})

